In [1]:
from cv2Utilities import *

In [ ]:
image = load_image(0)
show(warping(image, 1000))

# Some Parameters for better results

In [ ]:
image = load_image(1)
edge = canny(image, 5, 70)
dilated = dilate(edge, 7, 9)
eroded = erode(dilated, 7, 9)
cnt = contours(eroded, method=50)[0]
fin, approx = boundingRect(image, cnt)
show(fin)

## Drawing Circles on the origins

In [ ]:
for pnt in approx:
    cx, cy = pnt[0][0], pnt[0][1]
    cv2.circle(image, (cx, cy), 10, (0,0,255), 3)
show(image)

## Feature matching

In [ ]:
image1 = load_image(0)
gray1 = gray(image1)
orb = cv2.ORB_create(900)
keypoints1 = orb.detect(gray1, None)
keypoints1, descriptors1 = orb.compute(gray1, keypoints1)
print("Number of keypoints Detected: ", len(keypoints1))
show(cv2.drawKeypoints(image1, keypoints1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS))

### BRIEF with all details

In [ ]:
img1 = load_image(0)
gry1 = gray(img1)
img2 = load_image(2)
gry2 = gray(img2)

fast = cv2.FastFeatureDetector_create()
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

kp1 = fast.detect(gry1, None)
kp1, des1 = brief.compute(gry1, kp1)
kp2 = fast.detect(gry2, None)
kp2, des2 = brief.compute(gry2, kp2)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x:x.distance)
show(cv2.drawMatches(img1,kp1,img2,kp2,matches[:20], outImg=None, flags=2))

## ORB will all the gory details

In [ ]:
orb = cv2.ORB_create()

img1 = load_image(0)
img2 = load_image(1)
gry1 = gray(img1)
gry2 = gray(img2)

kp1, des1 = orb.detectAndCompute(gry1,None)
kp2, des2 = orb.detectAndCompute(gry2,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x:x.distance)

show(cv2.drawMatches(img1,kp1,img2,kp2,matches[:10], outImg=None, flags=2))

### BRIEF

In [16]:
img1 = load_image(2)
img2 = load_image(4)

show(BRIEF_compare(img1, img2, 50), name='BRIEF', done=False)
show(ORB_compare(img1, img2, nfeature=4000, n=50), name='ORB', done=False)
cv2.destroyAllWindows()